In [1]:
# import std libraries
import os
import time
import timeit
import pandas as pd
import numpy as np

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
# import FSA project-created libraries
from lib_UVMasterClean import UnifiedView_cleaning
from lib_mergeUV_NOMIS import uv_merge_nomis
from lib_mergeUV_CDRC import uv_merge_cdrc
import lib_UVDerive as uv_D
from lib_Options import Master_options
from lib_DeleteRubbish import delete_rubbish
from lib_Dummies import Master_dummies
from lib_NNT import NNTRegressor
from lib_RIO import train_RIO_model
from lib_RIO import RIO_predictor

os.makedirs('./csv', exist_ok=True)
os.makedirs('./temp', exist_ok=True)
keylist = '1_1_1_1_1_1_1_1'

In [3]:
# 1. clean the UV raw data [~20 min to run]

# identfy the file location of the raw data from UV and postcode dictionary:

# rawUVpath = 'Q:/Inform/Strategic Surveillance/Sprint AI/DATA/Unified_View/_business_data_fhrs-latest-inspection__view_detail.csv'
# rawUVpath = 'C:/Users/Neel.Savani-Patel/NEEL_tech/LOCAL_DATA/_business_data_fhrs-latest-inspection__view_detail.csv'
rawUVpath = 'C:/Users/Victor.SalasAranda/Desktop/FSA/csv/_business_data_fhrs-latest-inspection__view_detail.csv'

# ukpostcode = 'Q:/Inform/Strategic Surveillance/Sprint AI/DATA/ukpostcodes.csv'
# ukpostcode = 'C:/Users/Neel.Savani-Patel/NEEL_tech/LOCAL_DATA/ukpostcodes.csv'
ukpostcode = 'C:/Users/Victor.SalasAranda/Desktop/FSA/csv/ukpostcodes.csv'


# define file name of output .csv file
uv_clean_filename = './csv/UnifiedView_Master.csv'

pd_UV = UnifiedView_cleaning(rawUVpath, ukpostcode, uv_clean_filename)

In [4]:
# 2. merge with NOMIS    [~10 min to run - memory issue warning]

# using pd created above if already exist .....
# pd_UV_nomis = uv_nomis.uv_merge_nomis()

# else using the output .csv file created in cell above
UV_master_path = './csv/UnifiedView_Master.csv'

# nomis_path = 'Q:/Inform/Strategic Surveillance/Sprint AI/DATA/NOMIS_features/NOMIS_Features.csv'
# nomis_path = 'C:/Users/Neel.Savani-Patel/NEEL_tech/LOCAL_DATA/NOMIS_Features.csv'
nomis_path = 'C:/Users/Victor.SalasAranda/Desktop/FSA/csv/NOMIS_Features.csv'

# define file name of output .csv file
uvnomis_filename = './csv/UV_nomis_Master.csv'

pd_UV_nomis = uv_merge_nomis(UV_master_path, nomis_path, uvnomis_filename)

In [ ]:
# 3. merge with CDRC (~10min to run)

# using pd created above if already exist .....
# pd_UV_nomis = uv_nomis.uv_merge_nomis()

# else using the output .csv file created in cell above
uvnomis_path = './csv/UV_nomis_Master.csv'

# cdrc_path = 'C:/Users/Neel.Savani-Patel/NEEL_tech/LOCAL_DATA/CDRC_features.csv'
# cdrc_path = 'Q:/Inform/Strategic Surveillance/Sprint AI/DATA/CDRC_features/CDRC_features.csv'
cdrc_path = 'C:/Users/Victor.SalasAranda/Desktop/FSA/csv/CDRC_features.csv'

# define file name of output .csv file
uvcdrc_filename = './csv/UV_nomis_cdrc_Master.csv'

pd_UV_nomis = uv_merge_cdrc(uvnomis_path, cdrc_path, uvcdrc_filename)

In [ ]:
# 4. merge with derived data products (~4 min to run)

#%%  4a  SEASONALITY of inspection
uvcdrc_path = './csv/UV_nomis_cdrc_Master.csv'  # the output .csv file created in cell above
uvder_name01 = './csv/UV_derive1.csv' # define file name of output .csv file
pd_UV_der01 = uv_D.uv_seasonality(uvcdrc_path, uvder_name01)

# %%  4b  CHAIN RESTURANT (~4 min to run)
uv_path = './csv/UV_derive1.csv'
uvder_name02 = './csv/UV_derive2.csv'
pd_UV_der02 = uv_D.uv_chain(uv_path, uvder_name02)

In [ ]:
#5. DEFINE FULL AND TOTAL MASTERLIST OF EVERYTHING
MASTER_FINAL = './csv/uv_COMPLETE_master_FINAL.csv'
pd_UV_der02.to_csv(MASTER_FINAL)

In [ ]:
pd_UV_der02.head()

# Cull / Clean / Prepare for Neural Net
### perform in another TOPTREE script

In [ ]:
# 6. Select the options files [~2-3 min to run]

# Identfy the file location of the master dataset
# I_data_path = 'Q:/Inform/Strategic Surveillance/Sprint AI/DATA/Unified_View/UnifiedView_NOMIS_CDRC_Seasonality_ChainOrNot.csv'
I_data_path = './csv/uv_COMPLETE_master_FINAL.csv'

# Select the keys
# max. arguments = [3, 5, 6, 5, 4, 2, 4, 2]
I_keyarray = [1, 1, 1, 1, 1, 1, 1, 1]

# define file name of output .csv file
I_out_filename = 'master'

# function
keylist, Data6 = Master_options(I_data_path, I_keyarray, I_out_filename)

In [ ]:
# 7. Delete all the features from one csv  [~2-3 min to run]
# identfy the file location of the master dataset
I_pd_Masterlist = './csv/master_'+ keylist+ '.csv' 

# csv with the features deletion
I_csvfilename_of_features = './csv/Feature Deletion.csv'

# define file name of output .csv file
I_out_filename = 'clean_master_'+ keylist

# To see the delete list: (Options: 'Yes', 'No')
Bool = 'No' 

# function
Data7 = delete_rubbish(I_pd_Masterlist, I_csvfilename_of_features, I_out_filename, Bool)

In [ ]:
# 8. Create and add dummies features to the master csv (and delete categorical ones)  [~1-2 min to run]
# identfy the file location of the master dataset
I_data_path = './csv/clean_master_'+ keylist+ '.csv' 

# define file name of output .csv file
I_out_filename = 'master_dummies_'+ keylist

# Max number of dummies for each feature allowed (if ones have less that Max, those features will be removed)
Max = 50

# function
Data8 = Master_dummies(I_data_path, I_out_filename, Max)

In [ ]:
# 9. Create and train a NNT  [60~ min to run: it depends on how many models]
lastime=time.time()
# identfy the file location of the master dataset
I_data_path = './csv/master_dummies_'+ keylist+ '.csv' 

# Parameters
k = 4
num_epochs = 100
batch_Size = 128

# types of Models (Possibilities: 1 to 6, you can choose how many you want e.g: [2,3,1,6])
NNTList =[6]  

# Function
train_data, train_targets, test_data, test_targets = NNTRegressor(I_data_path, k, num_epochs, batch_Size, NNTList, keylist)
print((time.time()-lastime)/60)